### 사전준비

#### 필요한 라이브러리 설치

In [4]:
import sys, subprocess, importlib

print("Python:", sys.version)
print("Exe:", sys.executable)

# 1) pip이 없으면 ensurepip으로 부트스트랩
try:
    importlib.import_module("pip")
    print("pip already present.")
except ModuleNotFoundError:
    print("Bootstrapping pip via ensurepip...")
    subprocess.check_call([sys.executable, "-m", "ensurepip", "--upgrade"])

# 2) 이 커널의 파이썬에 직접 설치(경로 혼동 방지)
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip", "setuptools", "wheel"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas", "openpyxl", "numpy", "matplotlib"])

print("✅ 설치 완료")

Python: 3.13.5 (tags/v3.13.5:6cb20a2, Jun 11 2025, 16:15:46) [MSC v.1943 64 bit (AMD64)]
Exe: C:\Users\0215w\AppData\Local\Programs\Python\Python313\python.exe
pip already present.
✅ 설치 완료


### 전처리 수행

#### 데이터셋 불러오기

In [2]:
import os

# 구글드라이브 마운트용
#from google.colab import drive
#drive.mount('/content/drive')

DATA_DIR = r"/00_Sookmyunguniv/DACOS(2025)/데이터톤/데이터 분석/2025 숙명여대_물류센터 거래데이터_KEA"
# 혹시 코드 실제로 돌려보실 거면 저장 경로 로컬에 맞게 바꿔주세요!
OUT_DIR  = os.path.join(DATA_DIR, "_out")
os.makedirs(OUT_DIR, exist_ok=True)
print("DATA_DIR:", DATA_DIR)
print("OUT_DIR :", OUT_DIR)

DATA_DIR: /00_Sookmyunguniv/DACOS(2025)/데이터톤/데이터 분석/2025 숙명여대_물류센터 거래데이터_KEA
OUT_DIR : /00_Sookmyunguniv/DACOS(2025)/데이터톤/데이터 분석/2025 숙명여대_물류센터 거래데이터_KEA\_out


#### 데이터 통합

1. A센터 매입
2. A센터 매출
3. B센터 매입
4. B센터 매출

---

### 주요 컬럼 목록

##### 매입(purchase)

| 컬럼명             | 설명                       |
| --------------- | ------------------------ |
| 작업유형            | 매입 작업 유형(입고/반품 등)        |
| 일자              | 입고일(자동 변환)               |
| 매출처코드           | 점포/거래처 코드                |
| 매출처 우편번호        | 점포 우편번호                  |
| 공급업체 코드         | 공급사(매입처) 코드              |
| 공급업체 우편번호       | 공급사 우편번호                 |
| 입고 형태           | 직납/제3자 등(원본 기준)          |
| 상품코드            | 내부 상품 코드                 |
| 바코드             | 상품 바코드(대한상의 등)           |
| 상품명             | 상품명                      |
| 규격              | 규격(용량/사이즈 등)             |
| 옵션 코드           | 옵션 코드(판매 단위 등)           |
| 옵션              | 원본 옵션 값(EA/BOX/벌/낱개 등)   |
| 입수              | 한 박스당 EA 수(없으면 1 가정 가능)  |
| 수량              | 원본 수량(입고 수량)             |
| EA              | EA 기준 수량(원본 제공)          |
| 판매금액            | 매입 금액(원본 ‘판매금액’ 사용)      |
| 부가세             | 매입 부가세(‘부가세(과세)’ 포함해 통일) |
| 대분류             | 카테고리 대분류                 |
| 중분류             | 카테고리 중분류                 |
| 소분류             | 카테고리 소분류                 |
| (메타) 원본파일/센터/유형 | 각 행의 출처(선택적 저장)          |

---

##### 매출(sales)
| 컬럼명             | 설명                      |
| --------------- | ----------------------- |
| 판매일             | 판매일(자동 변환, 결측 허용)       |
| 구분              | 판매/반품 등 구분(결측 허용)       |
| 매출처코드           | 점포/거래처 코드               |
| 매출처 우편번호        | 점포 우편번호                 |
| 판매수량            | 원본 판매 수량                |
| 옵션코드            | 옵션 코드(판매 단위 등)          |
| 규격              | 규격                      |
| 입수              | 한 박스당 EA 수(없으면 1 가정 가능) |
| 바코드             | 상품 바코드                  |
| 상품명             | 상품명                     |
| 대분류             | 카테고리 대분류                |
| 중분류             | 카테고리 중분류                |
| 소분류             | 카테고리 소분류                |
| 공급가액            | 매출 공급가액(판매금액 없음)        |
| 부가세             | 매출 부가세                  |
| (메타) 원본파일/센터/유형 | 각 행의 출처(선택적 저장)         |

In [ ]:
# -*- coding: utf-8 -*-
# ✅ A/B 센터 × (매입/매출) → 4개의 CSV 생성
# - 파일명으로 A/B, 매입/매출 자동 분류
# - "매출처 이름"은 스키마에서 제거, 대신 "매출처 우편번호" 사용
# - 매입: 판매금액, 부가세 포함 / 매출: 공급가액, 부가세 포함 (판매금액 없음)
# - 매입의 '부가세(과세)'도 모두 '부가세'로 통일
# - 결과물: A센터_매입_통합.csv, A센터_매출_통합.csv, B센터_매입_통합.csv, B센터_매출_통합.csv

import os, re, glob
import numpy as np
import pandas as pd
from datetime import datetime

print("[경로확인] DATA_DIR =", DATA_DIR)
print("[경로확인] OUT_DIR  =", OUT_DIR)

# 결과 CSV에 행별 출처를 남기고 싶지 않다면 False (원본파일/센터/유형)
ADD_META = True

# -----------------------------
# 1) 표준 스키마 (매출처 이름 제거 반영)
# -----------------------------
# 매입(purchase): '매출처(이름)' 삭제, '매출처 우편번호' 채택, '부가세'만 사용
PURCHASE_TARGET = [
    "작업유형","일자","매출처코드","매출처 우편번호",
    "공급업체 코드","공급업체 우편번호","입고 형태",
    "상품코드","바코드","상품명","규격","옵션 코드","옵션",
    "입수","수량","EA","판매금액","부가세","대분류","중분류","소분류"
]

# 매출(sales): '매출처(이름)' 없음, '매출처 우편번호' 채택, 판매금액 없음(공급가액만)
SALES_TARGET = [
    "판매일","구분","매출처코드","매출처 우편번호",
    "판매수량","옵션코드","규격","입수","바코드","상품명",
    "대분류","중분류","소분류","공급가액","부가세"
]

# -----------------------------
# 2) 컬럼 별칭(원본 ↔ 표준 매핑)
# -----------------------------
def N(x):  # 정규화
    x = re.sub(r"\s+", "", str(x))
    x = re.sub(r"[()\[\]{}\-_/·•]", "", x)
    return x.lower()

ALIASES_PURCHASE = {
    "작업유형":         ["작업유형","작업 유형"],
    "일자":             ["일자","입고일자","입고 일자","기준일자","기준 일자"],
    "매출처코드":        ["매출처코드","소매점고유코드","거래처코드","매출처 코드"],
    "매출처 우편번호":     ["매출처우편번호","매출처 우편번호","우편번호","우 편 번 호"],
    "공급업체 코드":      ["공급업체코드","매입처코드","공급기업코드","매입처 코드"],
    "공급업체 우편번호":   ["공급업체우편번호","공급업체 우편번호","매입처우편번호","공급기업우편번호"],
    "입고 형태":         ["입고형태","입고 형태","입고_형태","입고분류","입고 분류"],
    "상품코드":          ["상품코드","품목코드","상품 코드","품목 코드"],
    "바코드":           ["바코드","상품바코드","상품 바코드","상품바코드대한상의","상품바코드(대한상의)"],
    "상품명":           ["상품명","품목명","상품 명","품 명"],
    "규격":             ["규격","규 격"],
    "옵션 코드":         ["옵션코드","옵션 코드","판매단위","판매 단위"],
    "옵션":             ["옵션","옵션명","옵션 명"],
    "입수":             ["입수","EA수","ea수"],
    "수량":             ["입고수량","입고 수량","수량"],
    "EA":              ["EA","ea"],
    "판매금액":          ["판매금액","공급가액","공급금액","금액"],  # 실제 엑셀에 '판매금액' 존재
    "부가세":           ["부가세","부가세(과세)","부 가 세(과세)","VAT","vat"],
    "대분류":           ["대분류","대 분류","분류대"],
    "중분류":           ["중분류","중 분류","분류중"],
    "소분류":           ["소분류","소 분류","분류소"],
}

ALIASES_SALES = {
    "판매일":           ["판매일","판매 일자","일자","기준일자","기준 일자"],
    "구분":             ["구분","매출구분","매출 반품구분","매출,반품구분"],
    "매출처코드":        ["매출처코드","소매점고유코드","거래처코드","매출처 코드"],
    "매출처 우편번호":     ["매출처우편번호","매출처 우편번호","우편번호","우 편 번 호"],
    "판매수량":          ["판매수량","수량","판매 수량"],
    "옵션코드":          ["옵션코드","옵션 코드","판매단위","판매 단위"],
    "규격":             ["규격","규 격"],
    "입수":             ["입수","EA수","ea수"],
    "바코드":           ["바코드","상품바코드","상품 바코드","상품바코드대한상의","상품바코드(대한상의)"],
    "상품명":           ["상품명","상품 명"],
    "대분류":           ["대분류","대 분류","분류대"],
    "중분류":           ["중분류","중 분류","분류중"],
    "소분류":           ["소분류","소 분류","분류소"],
    "공급가액":          ["공급가액","공급금액","공급 금액","금액"],
    "부가세":           ["부가세","부가세(과세)","VAT","vat"],
}

# -----------------------------
# 3) 변환 유틸
# -----------------------------
def build_col_map(df_cols, alias_book):
    cols_norm = {c: N(c) for c in df_cols}
    used = set(); result = {}
    for target, aliases in alias_book.items():
        key = None
        A = [N(a) for a in aliases]
        for src, sn in cols_norm.items():
            if src in used:
                continue
            if any(sn == a or a in sn or sn in a for a in A):
                key = src; used.add(src); break
        if key is not None:
            result[target] = key
    return result

def to_date(series):
    def _p(v):
        if pd.isna(v): return pd.NaT
        if isinstance(v, (pd.Timestamp, datetime)): return pd.to_datetime(v).date()
        if isinstance(v, (int, float)) and 20000 < float(v) < 60000:
            return (pd.to_datetime("1899-12-30") + pd.to_timedelta(int(v), unit="D")).date()
        for fmt in ("%Y-%m-%d","%Y/%m/%d","%Y.%m.%d","%Y%m%d","%y-%m-%d","%y/%m/%d"):
            try: return datetime.strptime(str(v).strip()[:10], fmt).date()
            except: pass
        return pd.to_datetime(v, errors="coerce").date()
    return series.apply(_p)

def to_num(series):
    return pd.to_numeric(series.astype(str).str.replace(",", "").str.strip(), errors="coerce")

def classify(path):
    name = os.path.basename(path)
    center = "A" if "A물류센터" in name else ("B" if "B물류센터" in name else None)
    kind   = "매입" if "매입" in name else ("매출" if "매출" in name else None)
    return center, kind

# -----------------------------
# 4) 파일 → 표준 스키마
# -----------------------------
def transform_one(path: str, center: str, kind: str) -> pd.DataFrame:
    print(f"\n===== 변환 시작: {os.path.basename(path)} | 센터={center} | 유형={kind} =====")
    df = pd.read_excel(path, engine="openpyxl")
    print(f"[원본 shape] {df.shape}")
    print("[원본 컬럼]", list(df.columns))

    if kind == "매출":
        aliases, target = ALIASES_SALES, SALES_TARGET
    else:
        aliases, target = ALIASES_PURCHASE, PURCHASE_TARGET

    # 1) 매핑 계산
    cmap = build_col_map(df.columns, aliases)
    print("\n[컬럼 매핑 결과] target -> source")
    for t in target:
        if t in cmap:
            print(f"  - {t} <= {cmap[t]}")
    missing = [t for t in target if t not in cmap]
    if missing:
        print("\n[주의] 원본에서 찾지 못한 타깃 컬럼 (빈 칼럼 생성):", missing)

    # 2) 타깃 스키마 빈 DF를 만들고, 매핑된 원본 컬럼을 개별 복사
    out = pd.DataFrame(index=df.index)
    for t in target:
        if t in cmap:
            out[t] = df[cmap[t]]
        else:
            out[t] = np.nan

    # 3) 타입 정리
    if kind == "매출":
        out["판매일"]   = to_date(out["판매일"])
        out["판매수량"] = to_num(out["판매수량"])
        out["입수"]     = to_num(out["입수"])
        out["공급가액"]  = to_num(out["공급가액"])
        out["부가세"]    = to_num(out["부가세"])
        out["구분"]      = out["구분"].astype("object")  # 결측 허용
        out = out[SALES_TARGET]  # 최종 순서 고정

    else:  # 매입
        out["일자"]     = to_date(out["일자"])
        out["입수"]     = to_num(out["입수"])
        out["수량"]     = to_num(out["수량"])
        out["EA"]       = to_num(out["EA"])
        out["판매금액"]   = to_num(out["판매금액"])
        out["부가세"]     = to_num(out["부가세"])  # '부가세(과세)' 포함 통일
        out = out[PURCHASE_TARGET]  # 최종 순서 고정

    # 4) 문자열 정리
    for c in out.columns:
        if out[c].dtype == "object":
            out[c] = out[c].astype(str).str.strip().replace({"nan": np.nan, "None": np.nan})

    # 5) 메타(선택)
    if ADD_META:
        out["원본파일"] = os.path.basename(path)
        out["센터"] = center
        out["유형"] = kind

    print(f"[변환 완료] shape={out.shape}")
    return out

# -----------------------------
# 5) 실행
# -----------------------------
files = sorted(glob.glob(os.path.join(DATA_DIR, "*.xlsx")))
print("\n[대상 파일]", len(files))
for f in files: print("  -", os.path.basename(f))

groups = {("A","매입"):[], ("A","매출"):[], ("B","매입"):[], ("B","매출"):[]}

for f in files:
    center, kind = classify(f)
    if center is None or kind is None:
        print(f"[건너뜀] 분류 불가: {os.path.basename(f)}")
        continue
    try:
        groups[(center, kind)].append(transform_one(f, center, kind))
    except Exception as e:
        print(f"[에러] {os.path.basename(f)} 처리 실패:", e)

save_as = {
    ("A","매입"): "A센터_매입_통합.csv",
    ("A","매출"): "A센터_매출_통합.csv",
    ("B","매입"): "B센터_매입_통합.csv",
    ("B","매출"): "B센터_매출_통합.csv",
}

for key, frames in groups.items():
    c, k = key
    if not frames:
        print(f"\n[알림] {c}센터 {k}: 병합할 데이터 없음")
        continue
    target = PURCHASE_TARGET if k == "매입" else SALES_TARGET
    merged = pd.concat(frames, ignore_index=True, sort=False)

    # 최종 컬럼 순서 강제 + 메타 부착
    cols = list(target)
    if ADD_META:
        cols += [x for x in ["원본파일","센터","유형"] if x in merged.columns]
    for t in cols:
        if t not in merged.columns: merged[t] = np.nan
    merged = merged[cols]

    out_path = os.path.join(OUT_DIR, save_as[key])
    merged.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"\n✅ 저장 완료: {out_path} | shape={merged.shape}")

print("\n🎉 완료. 로그의 매핑/누락 정보를 확인해 필요시 ALIASES_*만 보강하세요.")


[경로확인] DATA_DIR = /00_Sookmyunguniv/DACOS(2025)/데이터톤/데이터 분석/2025 숙명여대_물류센터 거래데이터_KEA
[경로확인] OUT_DIR  = /00_Sookmyunguniv/DACOS(2025)/데이터톤/데이터 분석/2025 숙명여대_물류센터 거래데이터_KEA\_out

[대상 파일] 6
  - A물류센터 매입 2021~2024 배포용.xlsx
  - A물류센터 매출 2021~2023 배포용.xlsx
  - A물류센터 매출 2024 배포용.xlsx
  - B물류센터 매입 2021~2024 배포용.xlsx
  - B물류센터 매출 2021~2023 배포용.xlsx
  - B물류센터 매출 2024 배포용.xlsx

===== 변환 시작: A물류센터 매입 2021~2024 배포용.xlsx | 센터=A | 유형=매입 =====
[원본 shape] (227534, 21)
[원본 컬럼] ['작업유형', '일자', '매출처코드', '매출처 우편번호', '공급업체 코드', '공급업체 우편번호', '입고 형태', '상품코드', '바코드', '상품명', '규격', '옵션 코드', '옵션', '입수', '수량', 'EA', '판매금액', '부가세(과세)', '대분류', '중분류', '소분류']

[컬럼 매핑 결과] target -> source
  - 작업유형 <= 작업유형
  - 일자 <= 일자
  - 매출처코드 <= 매출처코드
  - 매출처 우편번호 <= 매출처 우편번호
  - 공급업체 코드 <= 공급업체 코드
  - 공급업체 우편번호 <= 공급업체 우편번호
  - 입고 형태 <= 입고 형태
  - 상품코드 <= 상품코드
  - 바코드 <= 바코드
  - 상품명 <= 상품명
  - 규격 <= 규격
  - 옵션 코드 <= 옵션 코드
  - 옵션 <= 옵션
  - 입수 <= 입수
  - 수량 <= 수량
  - EA <= EA
  - 판매금액 <= 판매금액
  - 부가세 <= 부가세(과세)
  - 대분류 <= 대분류
  - 중분류 <= 중분

In [ ]:
#결측률 확인
import pandas as pd, os
for fn in ["A센터_매입_통합.csv","A센터_매출_통합.csv","B센터_매입_통합.csv","B센터_매출_통합.csv"]:
    p = os.path.join(OUT_DIR, fn)
    if os.path.exists(p):
        df = pd.read_csv(p, encoding="utf-8-sig")
        print("\n===", fn, "===")
        print(df.isna().mean().round(3))


=== A센터_매입_통합.csv ===
작업유형         0.000
일자           0.000
매출처코드        0.898
매출처 우편번호     0.898
공급업체 코드      0.000
공급업체 우편번호    0.000
입고 형태        0.000
상품코드         0.000
바코드          0.003
상품명          0.000
규격           0.002
옵션 코드        0.000
옵션           0.000
입수           0.000
수량           0.000
EA           0.000
판매금액         0.000
부가세          0.000
대분류          0.008
중분류          0.010
소분류          0.580
원본파일         0.000
센터           0.000
유형           0.000
dtype: float64


C:\Users\0215w\AppData\Local\Temp\ipykernel_14664\1408003209.py:6: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p, encoding="utf-8-sig")



=== A센터_매출_통합.csv ===
판매일         0.000
구분          0.000
매출처코드       0.000
매출처 우편번호    0.000
판매수량        0.000
옵션코드        0.000
규격          0.001
입수          0.000
바코드         0.002
상품명         0.000
대분류         0.003
중분류         0.157
소분류         0.584
공급가액        0.000
부가세         0.000
원본파일        0.000
센터          0.000
유형          0.000
dtype: float64

=== B센터_매입_통합.csv ===
작업유형         0.000
일자           0.000
매출처코드        0.408
매출처 우편번호     0.408
공급업체 코드      0.000
공급업체 우편번호    0.000
입고 형태        0.000
상품코드         0.000
바코드          0.000
상품명          0.000
규격           0.157
옵션 코드        0.000
옵션           0.000
입수           0.000
수량           0.000
EA           0.000
판매금액         0.000
부가세          0.000
대분류          0.000
중분류          0.677
소분류          0.688
원본파일         0.000
센터           0.000
유형           0.000
dtype: float64

=== B센터_매출_통합.csv ===
판매일         0.000
구분          0.000
매출처코드       0.000
매출처 우편번호    0.000
판매수량        0.000
옵션코드        0.000
규격          0

#### 결측치 보강

* (센터, 매출처코드) → 매출처 우편번호가 유일하게 관측된 경우에만 빈칸을 채움(여러 값이 섞이면 채우지 않고 리포트로 남김).

* 바코드 → 대/중/소도 각 레벨에서 유일한 값일 때만 채움(여러 값이 섞여 있으면 채우지 않음).

* 옵션코드/옵션/상품코드/규격/바코드/카테고리 등 코드·문자열 칼럼은 전부 문자열로 고정해 경고를 줄이고 재사용성을 높임.

* 우편번호는 숫자만 남기고 00000/000000 등 플레이스홀더는 결측으로 처리.


* (참고) 주요 컬럼 목록 (최종)
  
  *  매입(purchase)

      * 작업유형, 일자, 매출처코드, 매출처 우편번호, 공급업체 코드, 공급업체 우편번호, 입고 형태, 상품코드, 바코드, 상품명, 규격, 옵션 코드, 옵션, 입수, 수량, EA, 판매금액, 부가세, 대분류, 중분류, 소분류, (메타) 원본파일, 센터, 유형
  * 매출(sales)
      * 판매일, 구분, 매출처코드, 매출처 우편번호, 판매수량, 옵션코드, 규격, 입수, 바코드, 상품명, 대분류, 중분류, 소분류, 공급가액, 부가세, (메타) 원본파일, 센터, 유형


In [ ]:
# =========================
# 🔧 통합 CSV 후처리(보강) 셀
# - 위에서 만든 4개 CSV를 읽어서 결측을 안전하게 메워 수정본을 생성
# - 정책: 유일값만 채움(충돌 시 채우지 않고 리포트 저장)
# - 결과물: *_보강.csv (원본은 유지)
# =========================

import os
import numpy as np
import pandas as pd

# 0) 경로/파일명 (위 셀에서 DATA_DIR, OUT_DIR가 이미 정의되어 있다고 가정)
assert 'OUT_DIR' in globals(), "OUT_DIR 변수가 위 셀에서 정의되어 있어야 합니다."

file_map = {
    ("A","매입"): "A센터_매입_통합.csv",
    ("A","매출"): "A센터_매출_통합.csv",
    ("B","매입"): "B센터_매입_통합.csv",
    ("B","매출"): "B센터_매출_통합.csv",
}

def pjoin(*a): return os.path.join(OUT_DIR, *a)

# 1) 유틸 함수들
def as_str(s: pd.Series) -> pd.Series:
    return (s.astype(str).str.strip()
            .replace({"nan": np.nan, "None": np.nan, "": np.nan}))

def clean_zip(s: pd.Series) -> pd.Series:
    s = as_str(s).str.replace(r"\D", "", regex=True)
    return s.replace({"0": np.nan, "00000": np.nan, "000000": np.nan})

def load_csv_safe(path):
    # dtype은 일단 추론시키고, 아래에서 코드/문자열 칼럼만 명시적으로 문자열로 고정
    return pd.read_csv(path, encoding="utf-8-sig", low_memory=False)

# 2) 원본 4개 CSV 읽기
frames = {}
for key, fname in file_map.items():
    fpath = pjoin(fname)
    if os.path.exists(fpath):
        df = load_csv_safe(fpath)
        frames[key] = df
        print(f"[로드] {fname} shape={df.shape}")
    else:
        print(f"[경고] 파일 없음: {fpath}")

assert frames, "읽을 CSV가 없습니다."

# 3) 모두 합쳐 BIG 생성(마스터 만들 때 사용)
BIG = pd.concat(list(frames.values()), ignore_index=True, sort=False)

# 4-A) (센터, 매출처코드) → 매출처 우편번호 유일 매핑 만들기
def build_zip_master_unique(big: pd.DataFrame):
    use = big[["센터","매출처코드","매출처 우편번호"]].copy()
    use["매출처코드"] = as_str(use["매출처코드"])
    use["매출처 우편번호"] = clean_zip(use["매출처 우편번호"])
    use = use.dropna(subset=["매출처코드","매출처 우편번호"])

    g = use.groupby(["센터","매출처코드"])["매출처 우편번호"]
    nunique = g.nunique()

    # 유일값만 취함
    uniq_idx = nunique[nunique == 1].index
    zip_map = (g.first().loc[uniq_idx]
                 .reset_index()
                 .rename(columns={"매출처 우편번호":"zip_map"}))

    # 충돌 리포트 (둘 이상 값이 있는 키)
    conflict_idx = nunique[nunique > 1].index
    if len(conflict_idx) > 0:
        detail = (use.set_index(["센터","매출처코드"])
                      .loc[conflict_idx]
                      .reset_index()
                      .value_counts(["센터","매출처코드","매출처 우편번호"])
                      .reset_index(name="rows"))
    else:
        detail = pd.DataFrame(columns=["센터","매출처코드","매출처 우편번호","rows"])
    return zip_map, detail

zip_map, zip_conflicts = build_zip_master_unique(BIG)

# 4-B) 바코드 → (대/중/소) 유일 매핑 만들기
def build_cat_master_unique(big: pd.DataFrame):
    use = big[["바코드","대분류","중분류","소분류"]].copy()
    use["바코드"] = as_str(use["바코드"])
    use = use.dropna(subset=["바코드"])

    grp = use.groupby("바코드")
    def unique_or_nan(s):
        vals = s.dropna().unique()
        return vals[0] if len(vals) == 1 else np.nan

    cat_map = grp.agg({
        "대분류": unique_or_nan,
        "중분류": unique_or_nan,
        "소분류": unique_or_nan
    }).reset_index().rename(columns={
        "대분류":"대분류_map","중분류":"중분류_map","소분류":"소분류_map"
    })

    # 충돌 리포트(여러 값이 섞인 바코드들)
    conflicts = {}
    for col in ["대분류","중분류","소분류"]:
        choices = grp[col].agg(lambda s: list(pd.unique(s.dropna()))).reset_index()
        cf = choices[choices[col].apply(lambda x: len(x) > 1)].copy()
        cf = cf.rename(columns={col: f"{col}_choices"})
        conflicts[col] = cf
    return cat_map, conflicts

cat_map, cat_conflicts = build_cat_master_unique(BIG)

# 5) (선택) 충돌 리포트 저장
if not zip_conflicts.empty:
    zip_conflicts.to_csv(pjoin("리포트_우편번호_충돌키.csv"), index=False, encoding="utf-8-sig")
    print(f"[리포트] 우편번호 충돌키 저장: {pjoin('리포트_우편번호_충돌키.csv')}")
for k, dfc in cat_conflicts.items():
    if not dfc.empty:
        outp = pjoin(f"리포트_{k}_충돌바코드.csv")
        dfc.to_csv(outp, index=False, encoding="utf-8-sig")
        print(f"[리포트] {k} 충돌 바코드 저장: {outp}")

# 6) 각 파일에 보강 적용 + 문자열/우편번호 정리 + 저장( *_보강.csv )
def fix_and_save(df: pd.DataFrame, center: str, kind: str, fname: str):
    before = df[["매출처코드","매출처 우편번호","대분류","중분류","소분류"]].isna().mean()

    # 코드/문자열 칼럼 고정
    str_cols_common = ["매출처코드","매출처 우편번호","규격","바코드","상품명","대분류","중분류","소분류"]
    str_cols_sales  = ["옵션코드"]
    str_cols_purch  = ["상품코드","옵션 코드","옵션"]

    for c in str_cols_common:
        if c in df.columns: df[c] = as_str(df[c])
    if kind == "매출":
        for c in str_cols_sales:
            if c in df.columns: df[c] = as_str(df[c])
    else:
        for c in str_cols_purch:
            if c in df.columns: df[c] = as_str(df[c])

    # 우편번호 숫자 정리
    if "매출처 우편번호" in df.columns:
        df["매출처 우편번호"] = clean_zip(df["매출처 우편번호"])

    # (A) 우편번호 유일 매핑으로 보강 (원본이 NaN일 때만)
    if not zip_map.empty:
        df = df.merge(zip_map, on=["센터","매출처코드"], how="left")
        if "매출처 우편번호" in df.columns:
            df["매출처 우편번호"] = df["매출처 우편번호"].fillna(df["zip_map"])
        df.drop(columns=["zip_map"], inplace=True)

    # (B) 카테고리 유일 매핑으로 보강 (원본이 NaN일 때만)
    if not cat_map.empty:
        df = df.merge(cat_map, on="바코드", how="left")
        for col in ["대분류","중분류","소분류"]:
            df[col] = df[col].fillna(df[f"{col}_map"])
            df.drop(columns=[f"{col}_map"], inplace=True)

    after = df[["매출처코드","매출처 우편번호","대분류","중분류","소분류"]].isna().mean()
    print(f"\n[{center}센터 {kind}] 결측률 변화(전→후):")
    print(pd.concat([before.rename("before"), after.rename("after")], axis=1).round(3))

    # 저장(원본 유지, *_보강.csv 로 저장)
    name, ext = os.path.splitext(fname)
    out_name = f"{name}_보강{ext}"
    out_path = pjoin(out_name)
    df.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"✅ 저장: {out_path} | shape={df.shape}")

for (center, kind), df in frames.items():
    fname = file_map[(center, kind)]
    fix_and_save(df, center, kind, fname)

print("\n🎉 보강 완료. *_보강.csv 4개와(있다면) 충돌 리포트를 OUT_DIR에서 확인하세요.")


[로드] A센터_매입_통합.csv shape=(227534, 24)
[로드] A센터_매출_통합.csv shape=(1431051, 18)
[로드] B센터_매입_통합.csv shape=(345021, 24)
[로드] B센터_매출_통합.csv shape=(1541902, 18)
[리포트] 우편번호 충돌키 저장: /00_Sookmyunguniv/DACOS(2025)/데이터톤/데이터 분석/2025 숙명여대_물류센터 거래데이터_KEA\_out\리포트_우편번호_충돌키.csv
[리포트] 대분류 충돌 바코드 저장: /00_Sookmyunguniv/DACOS(2025)/데이터톤/데이터 분석/2025 숙명여대_물류센터 거래데이터_KEA\_out\리포트_대분류_충돌바코드.csv
[리포트] 중분류 충돌 바코드 저장: /00_Sookmyunguniv/DACOS(2025)/데이터톤/데이터 분석/2025 숙명여대_물류센터 거래데이터_KEA\_out\리포트_중분류_충돌바코드.csv
[리포트] 소분류 충돌 바코드 저장: /00_Sookmyunguniv/DACOS(2025)/데이터톤/데이터 분석/2025 숙명여대_물류센터 거래데이터_KEA\_out\리포트_소분류_충돌바코드.csv

[A센터 매입] 결측률 변화(전→후):
          before  after
매출처코드      0.898  0.898
매출처 우편번호   0.898  0.898
대분류        0.008  0.005
중분류        0.010  0.006
소분류        0.580  0.226
✅ 저장: /00_Sookmyunguniv/DACOS(2025)/데이터톤/데이터 분석/2025 숙명여대_물류센터 거래데이터_KEA\_out\A센터_매입_통합_보강.csv | shape=(227534, 24)

[A센터 매출] 결측률 변화(전→후):
          before  after
매출처코드      0.000  0.000
매출처 우편번호   0.000  0.000
대분류        0.003  0.001
중분류

In [3]:
# =========================
# 🔧 통합 CSV 후처리(보강) — 충돌 시 '최빈값' 사용
# - 위에서 만든 4개 CSV를 읽어서 결측을 메우고 *_보강.csv로 저장
# - 우편번호: (센터, 매출처코드) 별 최빈값으로 채움
# - 카테고리: (바코드) 별 대/중/소 각각 최빈값으로 채움
# - 원본은 유지, 같은 이름의 *_보강.csv는 덮어씀
# =========================

import os
import numpy as np
import pandas as pd

# 0) 경로/파일명 (위 셀에서 DATA_DIR, OUT_DIR가 이미 정의되어 있어야 함)
assert 'OUT_DIR' in globals(), "OUT_DIR 변수가 위 셀에서 정의되어 있어야 합니다."

file_map = {
    ("A","매입"): "A센터_매입_통합.csv",
    ("A","매출"): "A센터_매출_통합.csv",
    ("B","매입"): "B센터_매입_통합.csv",
    ("B","매출"): "B센터_매출_통합.csv",
}
def pjoin(*a): return os.path.join(OUT_DIR, *a)

# 1) 유틸 함수들 ---------------------------------------------------------------
def as_str(s: pd.Series) -> pd.Series:
    """문자열로 고정 + 공백/빈문자 정리"""
    return (s.astype(str).str.strip()
            .replace({"nan": np.nan, "None": np.nan, "": np.nan}))

def clean_zip(s: pd.Series) -> pd.Series:
    """우편번호: 숫자만 남기고 0/00000/000000 같은 플레이스홀더는 결측으로"""
    s = as_str(s).str.replace(r"\D", "", regex=True)
    return s.replace({"0": np.nan, "00000": np.nan, "000000": np.nan})

def mode_pick(s: pd.Series):
    """
    최빈값 선택(동률이면 값 사전순으로 결정해 일관성 유지).
    - NaN은 제외하고 계산
    """
    s = s.dropna()
    if s.empty:
        return np.nan
    vc = s.value_counts(dropna=True)  # desc 정렬
    maxc = vc.max()
    candidates = vc[vc == maxc].index
    # 동률이면 문자열 기준 사전순으로 하나 선택(결정 규칙 고정)
    return sorted([str(x) for x in candidates])[0] if len(candidates) > 1 else vc.index[0]

def load_csv_safe(path):
    return pd.read_csv(path, encoding="utf-8-sig", low_memory=False)

# 2) 원본 4개 CSV 읽기 ---------------------------------------------------------
frames = {}
for key, fname in file_map.items():
    fpath = pjoin(fname)
    if os.path.exists(fpath):
        df = load_csv_safe(fpath)
        frames[key] = df
        print(f"[로드] {fname} shape={df.shape}")
    else:
        print(f"[경고] 파일 없음: {fpath}")

assert frames, "읽을 CSV가 없습니다."

# 3) BIG(전체)에서 '최빈값' 마스터 만들기 ---------------------------------------
BIG = pd.concat(list(frames.values()), ignore_index=True, sort=False)

# (A) (센터, 매출처코드) → 매출처 우편번호 = 최빈값
zip_use = BIG[["센터","매출처코드","매출처 우편번호"]].copy()
zip_use["매출처코드"] = as_str(zip_use["매출처코드"])
zip_use["매출처 우편번호"] = clean_zip(zip_use["매출처 우편번호"])
zip_use = zip_use.dropna(subset=["매출처코드","매출처 우편번호"])

if not zip_use.empty:
    zip_map = (zip_use.groupby(["센터","매출처코드"])["매출처 우편번호"]
                      .agg(mode_pick)
                      .reset_index()
                      .rename(columns={"매출처 우편번호":"zip_map"}))
    print(f"[마스터] 우편번호 매핑 수: {len(zip_map)}")
else:
    zip_map = pd.DataFrame(columns=["센터","매출처코드","zip_map"])
    print("[마스터] 우편번호 매핑 생성 불가(원천 데이터 없음)")

# (B) 바코드 → (대/중/소) = 각 레벨 최빈값
cat_use = BIG[["바코드","대분류","중분류","소분류"]].copy()
cat_use["바코드"] = as_str(cat_use["바코드"])
cat_use = cat_use.dropna(subset=["바코드"])

if not cat_use.empty:
    cat_map = (cat_use.groupby("바코드")
                      .agg({"대분류":mode_pick, "중분류":mode_pick, "소분류":mode_pick})
                      .reset_index()
                      .rename(columns={"대분류":"대분류_map","중분류":"중분류_map","소분류":"소분류_map"}))
    print(f"[마스터] 카테고리 매핑 수: {len(cat_map)}")
else:
    cat_map = pd.DataFrame(columns=["바코드","대분류_map","중분류_map","소분류_map"])
    print("[마스터] 카테고리 매핑 생성 불가(원천 데이터 없음)")

# 4) 각 파일에 보강 적용 → *_보강.csv 저장 --------------------------------------
def fix_and_save(df: pd.DataFrame, center: str, kind: str, fname: str):
    # (가) 보강 전 결측률(핵심 칼럼) 출력
    before = df[["매출처코드","매출처 우편번호","대분류","중분류","소분류"]].isna().mean()

    # (나) 문자열/우편번호 정리
    str_cols_common = ["매출처코드","매출처 우편번호","규격","바코드","상품명","대분류","중분류","소분류"]
    str_cols_sales  = ["옵션코드"]
    str_cols_purch  = ["상품코드","옵션 코드","옵션"]
    for c in str_cols_common:
        if c in df.columns: df[c] = as_str(df[c])
    if kind == "매출":
        for c in str_cols_sales:
            if c in df.columns: df[c] = as_str(df[c])
    else:
        for c in str_cols_purch:
            if c in df.columns: df[c] = as_str(df[c])
    if "매출처 우편번호" in df.columns:
        df["매출처 우편번호"] = clean_zip(df["매출처 우편번호"])

    # (다) 우편번호: 최빈값 매핑으로 채우기 (원본이 NaN일 때만)
    if not zip_map.empty:
        df = df.merge(zip_map, on=["센터","매출처코드"], how="left")
        if "매출처 우편번호" in df.columns:
            df["매출처 우편번호"] = df["매출처 우편번호"].fillna(df["zip_map"])
        df.drop(columns=["zip_map"], inplace=True)

    # (라) 카테고리: 바코드 최빈값으로 채우기 (원본이 NaN일 때만)
    if not cat_map.empty:
        df = df.merge(cat_map, on="바코드", how="left")
        for col in ["대분류","중분류","소분류"]:
            df[col] = df[col].fillna(df[f"{col}_map"])
            df.drop(columns=[f"{col}_map"], inplace=True)

    # (마) 보강 후 결측률 비교 출력
    after = df[["매출처코드","매출처 우편번호","대분류","중분류","소분류"]].isna().mean()
    print(f"\n[{center}센터 {kind}] 결측률 변화(전→후):")
    print(pd.concat([before.rename("before"), after.rename("after")], axis=1).round(3))

    # (바) 저장(원본 유지, *_보강.csv)
    name, ext = os.path.splitext(fname)
    out_path = pjoin(f"{name}_보강{ext}")
    df.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"✅ 저장: {out_path} | shape={df.shape}")

for (center, kind), df in frames.items():
    fname = file_map[(center, kind)]
    fix_and_save(df, center, kind, fname)

print("\n🎉 보강 완료 — OUT_DIR에서 *_보강.csv 4개 파일을 확인하세요.")

[로드] A센터_매입_통합.csv shape=(227534, 24)
[로드] A센터_매출_통합.csv shape=(1431051, 18)
[로드] B센터_매입_통합.csv shape=(345021, 24)
[로드] B센터_매출_통합.csv shape=(1541902, 18)
[마스터] 우편번호 매핑 수: 2967
[마스터] 카테고리 매핑 수: 26083

[A센터 매입] 결측률 변화(전→후):
          before  after
매출처코드      0.898  0.898
매출처 우편번호   0.898  0.898
대분류        0.008  0.005
중분류        0.010  0.006
소분류        0.580  0.226
✅ 저장: /00_Sookmyunguniv/DACOS(2025)/데이터톤/데이터 분석/2025 숙명여대_물류센터 거래데이터_KEA\_out\A센터_매입_통합_보강.csv | shape=(227534, 24)

[A센터 매출] 결측률 변화(전→후):
          before  after
매출처코드      0.000  0.000
매출처 우편번호   0.000  0.000
대분류        0.003  0.001
중분류        0.157  0.002
소분류        0.584  0.155
✅ 저장: /00_Sookmyunguniv/DACOS(2025)/데이터톤/데이터 분석/2025 숙명여대_물류센터 거래데이터_KEA\_out\A센터_매출_통합_보강.csv | shape=(1431051, 18)

[B센터 매입] 결측률 변화(전→후):
          before  after
매출처코드      0.408  0.408
매출처 우편번호   0.408  0.408
대분류        0.000  0.000
중분류        0.677  0.646
소분류        0.688  0.658
✅ 저장: /00_Sookmyunguniv/DACOS(2025)/데이터톤/데이터 분석/2025 숙명여대_물류센터 거래데이

#### 분석 방향성 설정 및 필요한 컬럼만 떼어오기